# Apple - User Engagement with Artist Recommendations

```SQL
CREATE TABLE user_streams (
    stream_id INTEGER,
    user_id INTEGER,
    artist_id INTEGER,
    stream_date DATE
);

CREATE TABLE artist_recommendations (
    recommendation_id INTEGER,
    user_id INTEGER,
    artist_id INTEGER,
    recommendation_date DATE
);

INSERT INTO user_streams (stream_id, user_id, artist_id, stream_date)
VALUES
    (1, 101, 201, '2024-05-01'),
    (2, 102, 202, '2024-05-02'),
    (3, 101, 203, '2024-05-03'),
    (4, 103, 201, '2024-05-04'),
    (5, 104, 204, '2024-05-05'),
    (6, 105, 205, '2024-05-06'),
    (7, 102, 202, '2024-05-07'),
    (8, 101, 201, '2024-05-08'),
    (9, 106, 206, '2024-05-09'),
    (10, 107, 207, '2024-05-10'),
    (11, 108, 208, '2024-05-11'),
    (12, 109, 209, '2024-05-12'),
    (13, 110, 210, '2024-05-13'),
    (14, 101, 201, '2024-05-14'),
    (15, 102, 202, '2024-05-15'),
    (16, 103, 203, '2024-05-16'),
    (17, 104, 204, '2024-05-17'),
    (18, 105, 205, '2024-05-18'),
    (19, 106, 206, '2024-05-19'),
    (20, 107, 207, '2024-05-20');

INSERT INTO artist_recommendations (recommendation_id, user_id, artist_id, recommendation_date)
VALUES
    (1, 101, 201, '2024-05-01'),
    (2, 102, 202, '2024-05-02'),
    (3, 103, 203, '2024-05-03'),
    (4, 104, 204, '2024-05-04'),
    (5, 105, 205, '2024-05-05'),
    (6, 106, 206, '2024-05-06'),
    (7, 107, 207, '2024-05-07'),
    (8, 108, 208, '2024-05-08'),
    (9, 109, 209, '2024-05-09'),
    (10, 110, 210, '2024-05-10');
```

In [1]:
import pandas as pd
import numpy as np

In [6]:
df_artist = pd.read_csv('Data/009/artist_recommendations.csv')
df_streams = pd.read_csv('Data/009/user_streams.csv')

df_artist.head()

,recommendation_id,user_id,artist_id,recommendation_date
0,1,101,201,2024-05-01
1,2,102,202,2024-05-02
2,3,103,203,2024-05-03
3,4,104,204,2024-05-04
4,5,105,205,2024-05-05


In [3]:
df_streams.head()

,stream_id,user_id,artist_id,stream_date
0,1,101,201,2024-05-01
1,2,102,202,2024-05-02
2,3,101,203,2024-05-03
3,4,103,201,2024-05-04
4,5,104,204,2024-05-05


# Pregunta 1

### ¿Cuántos usuarios únicos han reproducido a un artista en la misma fecha o después de la fecha en que se les fue recomendado?

In [ ]:
df_conversion = pd.merge(df_artist, df_streams, on=['user_id','artist_id'])

df_exito = df_conversion[df_conversion['stream_date'] >= df_conversion['recommendation_date']]

total_usuarios = df_exito['user_id'].nunique()

total_usuarios

10

```SQL
SELECT
    count(distinct u.user_id)
FROM user_streams u
JOIN artist_recommendations a
    ON u.user_id = a.user_id
    AND u.artist_id = a.artist_id
WHERE u.stream_date >= a.recommendation_date;
```

# Pregunta 2

### ¿Cuál es el número promedio de veces que un artista recomendado es reproducido por los usuarios en mayo de 2024? Al igual que en la pregunta anterior, solo incluye las reproducciones realizadas en la misma fecha o después de la fecha en que el artista les fue recomendado.

In [10]:
df_combined = pd.merge(df_artist, df_streams, on=['user_id','artist_id'])

df_filtered = df_combined[
    (df_combined['stream_date'] >= df_combined['recommendation_date']) &
    (df_combined['stream_date'].between('2024-05-01','2024-05-31'))
]

streams_counts = df_filtered.groupby(['user_id','artist_id']).size()

resultado = streams_counts.mean()

resultado

np.float64(1.8)

```SQL
SELECT
    AVG(reproducciones_por_usuario)
FROM (SELECT u.user_id,
             u.artist_id,
             COUNT(*) AS reproducciones_por_usuario
      FROM user_streams u
               JOIN artist_recommendations a
                    ON u.user_id = a.user_id
                        AND u.artist_id = a.artist_id
      WHERE u.stream_date >= a.recommendation_date
        AND u.stream_date BETWEEN '2024-05-01' AND '2024-05-31'
      GROUP BY u.artist_id, u.user_id) AS conteos;
```

# Pregunta 3

### Entre los usuarios que escucharon al menos a un artista recomendado, ¿cuál es el promedio de artistas recomendados distintos que escucharon? Al igual que en la pregunta anterior, solo incluye las reproducciones que ocurrieron en la misma fecha o después de la fecha en que el artista le fue recomendado al usuario.

In [12]:
df_merged = pd.merge(df_artist, df_streams, on=['user_id','artist_id'])

df_valid = df_merged[
    (df_merged['stream_date'] >= df_merged['recommendation_date'])
]

conteo_por_usuario = df_valid.groupby('user_id')['artist_id'].nunique()

resultado = conteo_por_usuario.mean()

resultado

np.float64(1.0)

```SQL 
SELECT
    AVG(artistas_distintos_por_usuario) AS promedio_artistas_recomendados
FROM (
    SELECT
        u.user_id,
        COUNT(DISTINCT u.artist_id) AS artistas_distintos_por_usuario
    FROM user_streams u
    JOIN artist_recommendations r
        ON u.user_id = r.user_id
        AND u.artist_id = r.artist_id
    WHERE u.stream_date >= r.recommendation_date
    GROUP BY u.user_id
) AS subconsulta;

```